In [1]:
import rivapy
import datetime as dt
import math
import datetime as dt
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import numpy as np
import dateutil.relativedelta as relativedelta
from rivapy import enums

In [2]:
#Function to Rollout payment_dates
def payment_dates(trade_date,maturity_date,payment_cycle):
    date=maturity_date-relativedelta.relativedelta(months=+payment_cycle)
    payment_dates=[maturity_date]
    if maturity_date<date:
       payment_dates.append(maturity_date)
    while date>=trade_date:
        payment_dates.append(date)
        date=date-relativedelta.relativedelta(months=+payment_cycle)
    return sorted(payment_dates)
#payment_dates_ins=payment_dates(trade_date,maturity_date,payment_cycle)

In [3]:
#Define yield curve
object_id = "CDS_interest_rate_boots"
refdate = dt.datetime(2020,1,1)
days_to_maturity = [360, 720, 3*360, 4*360, 5*360, 7*360, 10*360,  15*360, 20*360,  30*360]
dates = [refdate + timedelta(days=d) for d in days_to_maturity]
rates = [0.002585,0.005034,0.008981,0.012954,0.016452,
                   0.021811,0.027007,0.031718,0.033834,0.035056]
#rates = [0,0,0,0,0,0,0,0,0,0]
dsc_fac = [math.exp(-rates[i]*days_to_maturity[i]/360) for i in range(len(days_to_maturity))]

        
dc_new = rivapy.marketdata.DiscountCurve(object_id, refdate, dates, dsc_fac, enums.InterpolationType.LINEAR, enums.ExtrapolationType.LINEAR)

In [6]:
mkt_par_spread=[0.0003,0.0009,0.0015,0.0021,0.0028,0.0043,0.0061,0.0063, 0.0068,0.0066]

refdate = dt.datetime(2020,1,1)
trade_date = refdate
RR=0.4
RR_rs=0.25
maturities_CDS_test = [12,24,36,48,60,7*12,10*12,15*12,20*12,30*12]

#Maturities and Payment Dates
#ref_date=dt.datetime(2020,1,1)
maturity_dates=[]
date=refdate
for i in range (len(maturities_CDS_test)):
    if maturities_CDS_test[i]>=12:
        date=refdate+relativedelta.relativedelta(years=(maturities_CDS_test[i]/12))
        maturity_dates.append(date)
    elif maturities_CDS_test[i]<12:
        date=refdate+relativedelta.relativedelta(month=maturities_CDS_test[i])
        maturity_dates.append(date)
    
maturity_dates_sorted=sorted(maturity_dates)

#Rollout instrument payment dates for different maturities
payment_dates_bootstrapp=[]
for i  in range(len(maturity_dates_sorted)):
    payment_dates_bootstrapp.append(payment_dates(refdate,maturity_dates_sorted[i],3))
payment_dates_bootstrapp=sorted(payment_dates_bootstrapp, key=lambda x: x[-1])

In [7]:
test=rivapy.marketdata.BootstrapHazardCurve(refdate, refdate, dc_new, RR, payment_dates_bootstrapp, mkt_par_spread)

In [8]:
test.calibrate_hazard_rate()

AttributeError: 'float' object has no attribute 'create_survival'

In [14]:
rivapy.marketdata.BootstrapHazardCurve(refdate, trade_date, dc_new, RR, payment_dates_bootstrapp, mkt_par_spread).calibrate_hazard_rate

<bound method BootstrapHazardCurve.calibrate_hazard_rate of <rivapy.marketdata.curves.BootstrapHazardCurve object at 0x00000210677A4F70>>